<img src="http://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png" align="right" width="30%">

# Estruturas para dados Multidimensionais

Nesta lição, abordamos os conceitos básicos das estruturas de dados em Xarray.
Ao final desta lição, você será capaz de:

- Compreender o básico sobre estruturas de dados em Xarray;
- Construir e inspecionar objetos `xarray.DataArray` e `xarray.Dataset`;
- Escrever e ler vários formatos de arquivos (por exemplo NetCDF, HDF5, Zarr) usando Xarray.

---

## Introdução

Arranjos multidimensionais (ou N-dimensionais, ND), frequentemente denominados tensores, são uma parte fundamental da computação científica.
Eles são encontrados nas mais diversas áreas, incluindo física, astronomia, geociências, bioinformática, engenharia, finanças e aprendizado de máquina (*Machine Learning*).
Em Python, [NumPy](https://numpy.org/) é um pilar fundamental para estruturas de dados, fornecendo a API (Interface de Programação de Aplicativos, do inglês *Application Programming Interface*) para trabalhar com arranjos brutos multidimensionais.
Entretando, conjuntos de dados do mundo real são normalmente mais do que simplesmente números brutos; eles envolvem rótulos que mapeiam esses arranjos numéricos com informações sobre a localização espacial, temporal, e outros.

Aqui está um exemplo sobre a estrutura do conjunto de dados empregados para a previsão do tempo:

<img src="http://xarray.pydata.org/en/stable/_images/dataset-diagram.png" align="center" width="80%">

Você vai reparar multiplas variáveis (temperatura, precipitação, informações sobre as coordenadas (latitude e longitude) e as dimensões (x, y, t). Logo vamos descobrir como toda essa informação se relaciona nas estruturas de dados em Xarray.

Xarray não apenas mantém um registro sobre todos os metadados envolvidos nos arranjos, ele também os usa para criar uma interface concisa e poderosa. Por exemplo:

- Aplica operações sobre dimensões pelo seu nome: `x.sum('time')`;

- Seleciona valores pela etiqueta (ou localização lógica) em vez de pela contagem de índice inteiro: `x.loc['2014-01-01']` ou `x.sel(time='2014-01-01')`.

- Operações matemáticas (i.e., `x - y`) vetorizadas e expandidas para multiplas dimensões (propagação do arranjo, ou *array broadcasting*) baseados no nome da dimensão, e não em sua forma.

- Use facilmente o paradígma *separar-aplicar-combinar* com `groupby`: `x.groupby('time.dayofyear').mean()`.

- Alinhamento dos arranjos baseados nos rótulos semelhantes, que funcionam facilmente para valores ausentes: `x, y = xr.align(x, y, join='outer')`.

- Mantém o registro de metadados arbitrários na corma de um dicionário Python: `x.attrs`.

A naturesa N-dimensional das estruturas de dados xarray as tornam aplicáveis para lidar com dados científicos multi-dimensionais, além de que utilizar o nome das dimensões ao invés da numeração dos eixos (`dim='time'` ao invés de `axis=0`) torna tais estruturas muito mais maleáveis do que os arranjos de dados brutos numpy (`ndarray`): com xarray, você não precisa memorizar a ordem das dimensões dos tensores ou inserir uma dimensão postiça de tamanho 1 para alinhas os tensores (i.e., usando `np.newaxis`).

O ganho imediato ao utilizar xarray é que você vai escrever menos código. A longo prazo, o retorno é que você conseguirá compreender facilmente o que estava pensando quando revisitar um código semanas ou meses após programa-lo.


## Estruturas de dados

Xarray fornece duas estruturas de dados: `DataArray` e `Dataset`.
A classe `DataArray` anexa a nomenclatura das dimensões, coordenadas e atributos dos arranjos multi-dimensionais, enquanto `Dataset` combina diversos arranjos em uma única estrutura.

Ambas classes são usualmente criadas quando lemos dados do disco, mas para melhor compreende-las, vamos primeiro ver o código necessário para cria-las.

### DataArray

A classe `DaraArray` é utilizada para anexar o nome do arranjo, das dimensões e rótudos, além de seus atributos.

Por exemplo, vamos criar um `DataArray` denominado `a` com três dimensões (são elas `x`, `y` e `z`) a partir de um arranjo numpy:

In [ ]:
import numpy as np
import xarray as xr

rng = np.random.default_rng(seed=0)  # Vamos discutir isso depois

In [ ]:
da = xr.DataArray(
    np.ones((3, 4, 2)),
    dims=("x", "y", "z"),
    name="a",
    coords={"z": [-1, 1], "u": ("x", [0.1, 1.2, 2.3])},
    attrs={"attr": "value"},
)

Nesse caso, usamos um arranjo `numpy` 3x4x2 com todos os valores definidos como `1`, mas note que podemos usar qualquer coisa que ao menos se comporte de maneira semelhante a um arranjo `numpy` ou que possa ser transformado em um arranjo [`numpy.array`](https://numpy.org/doc/stable/reference/generated/numpy.array.html).

Nós informamos ao argumento `dims` uma sequência (uma tupla nessa caso, mas também poderia ter sido uma lista) contendo o nome das dimensões `x`, `y` e `z`.
No caso de uma única coordenada, poderíamos ter informado apenas o seu nome com oargumento. Por exemplo:

```python
xr.DataArray([1, 1], dims="x")
```

O name das coordenadas (e o nome `name` do arranjo) podem ser qualquer coisa que funcione em um `set` Python (i.e, envocar `hash()` neles não dispare um erro), mas para serem úteis, o ideal é utilizar `strings`.

`coords` são uma estrutura [semelhantes com um dicionário](https://docs.python.org/pt-br/3/glossary.html#term-mapping) Python, que mapeia os valores (coordenadas) correspondente a cada ponto da dimensão (e.g., um vetor numérico, objetos com tempo/data ou mesmo texto). Veremos com mais detalhes ao longo do curso.

Podemos também anexar ao `DataArray` qualquer informação relevante como metadados (atributos) ao fornecer um dicionário ao parâmetro `attrs`.

#### Representação visual

Agora que definimos nosso primeiro `DataArray`, nós podemos visualiza-lo em sua representação textual.

Xarray tem dois tipos de representação: `html` (que está disponível apenas nos *notebooks*) e `texto`. Para escolher entre ambas, usamos a opção `display_style`.

Começamos com a representação textual:

In [ ]:
with xr.set_options(display_style="text"):
    display(da)

Ela consiste em:

- O nome do arranjo `DataArray` (`'a'`). Se não fornecermos essa informação, ela será omitida da representação;
- As dimensões do arranjo `(x: 3, y: 4, z: 2)`: Isso nos diz que a primeira dimensão é denominada `x` e tem o tamanho `3`, a segunda dimensão é `y` com o tamanho `4`, e a terceira dimensão é `z` com o tamanho `2`;
- Uma prévia dos valores;
- Uma listagem (não ordenada) das coordenadas ou dimensões com coordenadas com um item por linha. Cada item tem seu nome, uma ou mais dimensões antes parenteses, um `dtype` a uma prévia dos valores. Note que haverá uma marcação `*` nos itens que são também uma dimensão;
- Uma listagem em orda alfabética das dimensões sem coordenadas;
- Uma lista (não ordenada) dos atributos.

A representação `html` tem aparência similar:


In [ ]:
with xr.set_options(display_style="html"):
    display(da)

Mas note que a prévia dos valores foi condensada para uma única linha (pode ser expandida ao clicar no sómbolo à esquerda) e as dimensões são marcadas em negrito em vez do prefixo `*`.

Ao longo do material, vamos manter a representação HTML exceto quando explicando a notação textual.


Uma vez que criamos o `DataArray`, podemos visualizar todas as informações contidas com:


In [ ]:
da.data

In [ ]:
da.dims

In [ ]:
da.coords

In [ ]:
da.attrs

#### Coordenadas

Como mencionado anteriormente, `coords` é um container similar à um dicionário que mapeia nomes para valores.
Ele pode ser:

- Outro objeto `DataArray`;
- Uma tupla na forma `(dims, data, attrs)`, onde `attrs` é opcional. Isso é na verdade equivalente a criação de um novo objeto com `DataArray(dims=dims, data=data, attrs=attrs)`;
- Um arranjo `numpy` (ou qualquer coisa que possa ser convertida em um usando `numpy.array`).

Vejamos um novo exemplo:


In [ ]:
da = xr.DataArray(
    np.ones((3, 4)),
    dims=("x", "y"),
    coords={
        "x": ["a", "b", "c"],
        "y": np.arange(4),
        "u": ("x", np.arange(3), {"attr1": 0}),
    },
)
da

Vemos que designamos o mapeamento para as dimensões `x` e `y` e também criamos uma coordenada denominada `u` ao longo de `x` com seus próprios metadados (clique no ícone da planilha para visualizar).

A diferença entre o mapeamento de dimensões (dimensões coordenadas) e uma corrdenada normal é que, por hora, as operações com indexação (`sel`, `reindex`, etc) apenas estão disponíveis para dimensões coordenadas. Note também, enquanto coordenadas podem ter uma dimensão arbitrária, dimensões coordenadas devem ser unidimensionais.


#### Exercícios

Crie um `DataArray` denominado "height" a partir da números aleatórios:

1. Com as dimensões denominadas "latitude" e "longitude"


In [ ]:
height = rng.random((180, 360)) * 400
xr.DataArray(
    # Seu código aqui
)

2. Com as dimensões coordenadas:

- "latitude": de -90 a 90 com passo 1
- "longitude": de -180 a 180 com passo 1


In [ ]:
xr.DataArray(
    # Seu código aqui
)

3. Com metadados tanto para os dados quando para as coordenadas:

- height: "type": "ellipsoid"
- latitude: "type": "geodetic"
- longitude: "prime_meridian": "greenwich"


In [ ]:
xr.DataArray(
    # your code here
)

### Dataset

Objetos `Dataset` podem conter múltiplas variáveis, cada uma possivelmente com diferentes dimensões.

A contrução de um `Dataset` aceita três argumentos:

- `data_vars`: Dicionário que mapeia nomes à valores. Tem o formato simular ao descrito em [coordenadas](#Coordenadas), exceto que deve ser um objeto `DataArray` ou a sintaxe com tuplas, já que temos que fornecer as dimensões.
- `coords`: Mesmo que para `DataArray`.
- `attrs`: Mesmo que para `Dataset`.

Por exemplo, vamos criar um `Dataset` com duas variáveis:


In [ ]:
ds = xr.Dataset(
    data_vars={
        "a": (("x", "y"), np.ones((3, 4))),
        "b": ("t", np.full((8,), 3), {"attr": "value"}),
    },
    coords={
        "x": [-1, 0, 1],
    },
    attrs={"attr": "value"},
)

#### Representação visua

Novamente, vemos primeiro a representação textual:


In [ ]:
with xr.set_options(display_style="text"):
    display(ds)

Ela consiste em

- Um descrição de todas as dimensões no `dataset` e seus comprimentos;
- Uma lista das coordenadas (mesmo formata que para `DataArray`);
- Uma lista das dimensões sem coordenadas;
- Uma lista das variáveis armazenadas.

Agora, a representação HTML:


In [ ]:
with xr.set_options(display_style="html"):
    display(ds)

#### Coordenadas

Assim como para `DataArray`, um conjunto `Dataset` se torna mais útil quando designamos coordenadas.
Aqui podemos também exemplificar o uso de um objeto `datetime` Pandas como mapeamento de coordenadas:


In [ ]:
import pandas as pd

xr.Dataset(
    data_vars={
        "a": (("x", "y"), np.ones((3, 4))),
        "b": (("t", "x"), np.full((8, 3), 3)),
    },
    coords={
        "x": ["a", "b", "c"],
        "y": np.arange(4),
        "t": pd.date_range("2020-07-05", periods=8, freq="D"),
    },
)

No caso det ermos variáveis com valores diferentes ao longa de uma mesma dimensão, não podemos mais usar a sintaxe reduzida apresentada acima. Em vez disso, temos que usar objetos `DataArray`:


In [ ]:
x_a = np.arange(1, 4)
x_b = np.arange(-1, 3)

a = xr.DataArray(np.linspace(0, 1, 3), dims="x", coords={"x": x_a})
b = xr.DataArray(np.zeros(4), dims="x", coords={"x": x_b})

xr.Dataset(data_vars={"a": a, "b": b})

que combina as coordenadas e preenche os espaços vazios com com `nan` (convertendo os dados para ponto flutuante no processo). Por exemplo, `b` não tem um valor para `x==3`, então `nan` foi usado em seu lugar.


#### Exercícios

1. Crie um Dataset com duas variáveis ao longo de `latitude` e `longitude`: `height` e `gravity_anomaly`


In [ ]:
height = rng.random((180, 360)) * 400
gravity_anomaly = rng.random((180, 360)) * 400 - 200

xr.Dataset(
    # Seu código aqui
)

2. Adicione as coordenadas `latitude` e `longitude`:

- `latitude`: de -90 até 90 com passo 1
- `longitude`: de -180 até 180 com passo 1


In [ ]:
xr.Dataset(
    # Seu código aqui
)

3. Adicione metadados para coordenadas a variáveis:

- `latitude`: "type": "geodetic"
- `longitude`: "prime_meridian": "greenwich"
- `height`: "ellipsoid": "wgs84"
- `gravity_anomaly`: "ellipsoid": "grs80"


In [ ]:
xr.Dataset(
    # Seu código aqui
)

## Conversões e I/O

Os objetos `DataArray` e `Dataset` são muito frequentemente criados por meio de conversões com outras bibliotecas Python, por exemplo [pandas](https://pandas.pydata.org/), ou ao ler informação armazenada em formatos como [NetCDF](https://www.unidata.ucar.edu/software/netcdf/) ou
[zarr](https://zarr.readthedocs.io/en/stable/).

Para converter de/para `pandas`, podemos usar o método <code>[to_xarray](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_xarray.html)</code> contido nos objetos [pandas](https://zarr.readthedocs.io/en/stable/), ou ainda o método <code>[to_pandas](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.to_pandas.html)</code> contido em objetos `xarray`.


In [ ]:
series = pd.Series(np.ones((10,)), index=list("abcdefghij"))
series

In [ ]:
arr = series.to_xarray()
arr

In [ ]:
arr.to_pandas()

Podemos ainda controlar qual objeto `pandas` é utilizado ao invocar `to_series` ou `to_dataframe`:


In [ ]:
ds = xr.Dataset(
    data_vars={"a": ("x", np.arange(5)), "b": (("x", "y"), np.ones((5, 4)))}
)

**<code>to_series</code>**: Sempre irá converter objetos `DataArray` para `pandas.Series`, usando `MultiIndex` para lidar com multiplas dimensões


In [ ]:
ds.a.to_series()

In [ ]:
ds.b.to_series()

**<code>to_dataframe</code>**: Sempre irá converter objetos `DataArray` ou `Dataset`
para `pandas.DataFrame`. Note que objetos `DataArray` devem ser nomeados para que isso funcione.


In [ ]:
ds.a.to_dataframe()

Uma vez que as colunas em um `DataFrame` devem ter os mesmos índices, eles são automaticamente difundidos.


In [ ]:
ds.to_dataframe()

### I/O

Um dos recursos mais usados do Xarray é a capacidade de ler e escrever
para uma variedade de formatos de dados. Por exemplo, o Xarray pode ler o seguinte
formatos:

- [NetCDF](https://www.unidata.ucar.edu/software/netcdf/) / GRIB (via
  `open_dataset` / `open_mfdataset`, `to_netcdf` / `save_mfdataset`)
- [Zarr](https://zarr.readthedocs.io/en/stable/) (via `open_zarr`, `to_zarr`)
- [GeoTIFF](https://gdal.org/drivers/raster/gtiff.html) /
  [GDAL rasters](https://svn.osgeo.org/gdal/tags/gdal_1_2_5/frmts/formats_list.html)
  (via `open_rasterio`)

#### NetCDF

A maneira recomendada de armazenar estruturas de dados xarray é NetCDF, que é um formato de arquivos binários para conjuntos de dados autodescritos (que se originaram nas geociências).
Xarray é baseado no modelo de dados netCDF, então arquivos netCDF no disco diretamente
correspondem a objetos `Dataset`.

O Xarray lê e grava em arquivos NetCDF usando o as funções `open_dataset`/`open_dataarray` e o método `to_netcdf`.

Vamos primeiro criar alguns conjuntos de dados e gravá-los no disco usando `to_netcdf`, que
segue o caminho para o qual queremos escrever:

In [ ]:
ds1 = xr.Dataset(
    data_vars={
        "a": (("x", "y"), np.random.randn(4, 2)),
        "b": (("z", "x"), np.random.randn(6, 4)),
    },
    coords={
        "x": np.arange(4),
        "y": np.arange(-2, 0),
        "z": np.arange(-3, 3),
    },
)
ds2 = xr.Dataset(
    data_vars={
        "a": (("x", "y"), np.random.randn(7, 3)),
        "b": (("z", "x"), np.random.randn(2, 7)),
    },
    coords={
        "x": np.arange(6, 13),
        "y": np.arange(3),
        "z": np.arange(3, 5),
    },
)

# Escreve os datasets
ds1.to_netcdf("ds1.nc")
ds2.to_netcdf("ds2.nc")

# Escreve dataarray
ds1.a.to_netcdf("da1.nc")

Reading those files is just as simple:


In [ ]:
xr.open_dataset("ds1.nc")

In [ ]:
xr.open_dataarray("da1.nc")

#### Zarr

[Zarr](https://zarr.readthedocs.io/en/stable/) é um pacote Python e um formato de dados que fornece uma implementação de matrizes N-dimensionais em partes (*chunked*).
Zarr tem a capacidade de armazenar matrizes de várias maneiras, incluindo na memória, em
arquivos e em armazenamento de objeto baseado em nuvem, como Amazon S3 e Google Cloud
Armazenamento. O back-end Zarr do Xarray permite que o xarray aproveite esses recursos.

Os arquivos Zarr podem ser gravados com:


In [ ]:
ds1.to_zarr("ds1.zarr", mode="w")

ou para qualquer interface `MutableMapping`:

In [ ]:
mystore = {}

ds1.to_zarr(store=mystore)

Podemos então ler o arquivo criado com:


In [ ]:
xr.open_zarr("ds1.zarr", chunks=None)

definir o parâmetro `chunks` para` None` evita `dask` (mais sobre isso a seguir).
